
# Kraken + ALTO OCR — Clean Colab (Non‑expert friendly)

This notebook helps you **train and evaluate** a Kraken recognition model on ALTO XML data using Google Colab.

**What’s improved here**  
- Fast setup: we **don’t** install large packages on Drive. Instead, we use a **small pip cache on Drive** to make future installs much faster.  
- Single place to edit paths: there’s **one Project Settings** cell with easy form fields.  
- Step‑by‑step explanations with clear instructions and checks.  
- Removed unused/debug cells.

> Tip: Run cells **top to bottom**. Each cell explains what it does.



## 1) Connect Google Drive

This mounts your Drive at `/content/drive` so we can store training data, models, and the small pip cache.


In [ ]:

from google.colab import drive  # type: ignore
drive.mount('/content/drive')
print("✅ Drive mounted at /content/drive")



## 2) Project Settings (edit here once)

Fill these fields. We will create folders automatically if they don't exist.

- `PROJECT_ID`: a short name for your project (e.g., `0093`).  
- `DATA_DIR`: where your ALTO data lives. It should contain **images** and **ALTO XMLs** in a structure your lists expect.  
- `MODELS_DIR`: where trained models will be stored.  
- `CPU_THREADS`: how many CPU threads to use.  
- `DEVICE`: `"cpu"` is safe for everyone. If you enable GPU in Colab (**Runtime → Change runtime type → T4**), you can try `cuda`.

> You can keep the defaults below and only change `PROJECT_ID`.


In [ ]:

#@title 🔧 Project Settings
from pathlib import Path
import os

PROJECT_ID = "0093"  #@param {type:"string"}
ROOT_IN_DRIVE = "/content/drive/MyDrive"

# Where your ALTO data is (images + XML). Change if needed.
DATA_DIR = f"{ROOT_IN_DRIVE}/kraken_data/{PROJECT_ID}"  #@param {type:"string"}
# Where models should be saved
MODELS_DIR = f"{ROOT_IN_DRIVE}/kraken_models/{PROJECT_ID}/rec"  #@param {type:"string"}
# Small persistent pip cache (fast re-installs without big space usage)
PIP_CACHE_DIR = f"{ROOT_IN_DRIVE}/.pip-cache"  #@param {type:"string"}

# Training config
CPU_THREADS = 2  #@param {type:"number"}
DEVICE = "cpu"   #@param ["cpu", "cuda"]

# Derived paths (do not edit)
Path(DATA_DIR).mkdir(parents=True, exist_ok=True)
Path(MODELS_DIR).mkdir(parents=True, exist_ok=True)
Path(PIP_CACHE_DIR).mkdir(parents=True, exist_ok=True)

# File lists (we'll create/check these later)
LISTS_DIR = "/content/lists"
Path(LISTS_DIR).mkdir(parents=True, exist_ok=True)
TRAIN_LIST = f"{LISTS_DIR}/train.txt"
VAL_LIST   = f"{LISTS_DIR}/val.txt"

print("✅ Settings applied")
print("PROJECT_ID:", PROJECT_ID)
print("DATA_DIR:", DATA_DIR)
print("MODELS_DIR:", MODELS_DIR)
print("PIP_CACHE_DIR:", PIP_CACHE_DIR)
print("LISTS_DIR:", LISTS_DIR)



## 3) Install Kraken (fast) — only if missing

Colab machines are temporary. We can't permanently install system tools on Google’s side, but we can **speed up installs** by using a **small pip cache on Drive**.  
- First time: takes a few minutes.  
- Next times (with same account): much faster because cached wheels are re-used.

This cell **checks if Kraken is already available**. If not, it installs it.


In [ ]:

import os, sys, subprocess, shlex

# Persist pip cache on Drive (small space, big speed-up)
os.environ["PIP_CACHE_DIR"] = PIP_CACHE_DIR

def is_importable(pkg: str) -> bool:
    try:
        __import__(pkg)
        return True
    except Exception:
        return False

need_install = not is_importable("kraken")

if not need_install:
    import kraken
    try:
        import torch
        torch_v = torch.__version__
    except Exception:
        torch_v = "unknown"
    print(f"✅ Kraken already available (version: {getattr(kraken, '__version__', 'unknown')}) | Torch: {torch_v}")
else:
    print("⏳ Installing Kraken (this is only needed the first time on a new Colab VM)...")
    # Upgrade pip first (often speeds up dependency resolution)
    subprocess.run(shlex.split("python -m pip -q install --upgrade pip"), check=True)
    # Install kraken (cairo extras enable PDF/ALTO pipelines)
    # Torch will be pulled automatically with a compatible build
    subprocess.run(shlex.split("python -m pip -q install 'kraken[cairo]'"), check=True)

    # Verify
    import kraken, importlib
    importlib.reload(kraken)
    try:
        import torch
        torch_v = torch.__version__
    except Exception:
        torch_v = "unknown"
    print(f"✅ Installed Kraken (version: {getattr(kraken, '__version__', 'unknown')}) | Torch: {torch_v}")



## 4) Create or Validate Training Lists

Kraken’s `ketos train` expects lists of image/XML pairs.  
If you **already have** `train.txt` and `val.txt`, place them in `/content/lists` and skip generation.  
Otherwise, use the helper below to **auto-generate lists** by scanning `DATA_DIR`.

**Expected format (one sample per line):**  
```
/path/to/image.png	/path/to/alto.xml
```


In [ ]:

# Helper: build simple lists by pairing image files and ALTO xmls with same stem.
# Adjust patterns if your data layout differs.

import os
from pathlib import Path
from typing import List, Tuple

IMG_EXTS = {".png", ".jpg", ".jpeg", ".tif", ".tiff"}
XML_EXTS = {".xml"}

def find_pairs(root: str) -> List[Tuple[str, str]]:
    rootp = Path(root)
    imgs = {}
    xmls = {}
    for p in rootp.rglob("*"):
        if p.suffix.lower() in IMG_EXTS:
            imgs[p.stem] = str(p)
        elif p.suffix.lower() in XML_EXTS:
            # Heuristic: only include ALTO-like files (contains <alto> tag)
            try:
                # Fast check without fully parsing (safe for small files)
                with open(p, "r", encoding="utf-8", errors="ignore") as fh:
                    head = fh.read(4096)
                    if "<alto" in head:
                        xmls[p.stem] = str(p)
            except Exception:
                pass
    pairs = []
    for k, img in imgs.items():
        if k in xmls:
            pairs.append((img, xmls[k]))
    return pairs

def write_list(pairs: List[Tuple[str, str]], out_path: str):
    with open(out_path, "w", encoding="utf-8") as f:
        for img, xml in pairs:
            f.write(f"{img}\t{xml}\n")

AUTO_GENERATE = True  # change to False if you already prepared lists

if AUTO_GENERATE:
    pairs = find_pairs(DATA_DIR)
    n = len(pairs)
    if n < 2:
        raise SystemExit(f"❌ Not enough samples in {DATA_DIR}. Found {n} image+ALTO pairs. "
                         f"Please check your data structure.")
    # simple split 90/10
    cut = max(1, int(n * 0.9))
    train_pairs, val_pairs = pairs[:cut], pairs[cut:]
    write_list(train_pairs, TRAIN_LIST)
    write_list(val_pairs, VAL_LIST)
    print(f"✅ Auto-generated lists: {len(train_pairs)} train, {len(val_pairs)} val")
else:
    # Validate existing lists
    for p in [TRAIN_LIST, VAL_LIST]:
        if not Path(p).exists():
            raise SystemExit(f"❌ Expected list not found: {p}")
    print("✅ Using existing train/val lists.")



## 5) Sanity Check

We count samples and show a few example lines so you can quickly verify the lists are correct.


In [ ]:

from itertools import islice

def count_lines(p):
    with open(p, 'r', encoding='utf-8') as f:
        return sum(1 for _ in f)

def head(p, n=5):
    with open(p, 'r', encoding='utf-8') as f:
        return list(islice(f, n))

n_train = count_lines(TRAIN_LIST) if Path(TRAIN_LIST).exists() else 0
n_val = count_lines(VAL_LIST) if Path(VAL_LIST).exists() else 0

print(f"Train samples: {n_train} | Val samples: {n_val}")
print("\nSample train lines:")
print("".join(head(TRAIN_LIST, 5)))
print("Sample val lines:")
print("".join(head(VAL_LIST, 3)))

if n_train == 0:
    raise SystemExit("❌ No training samples found. Please fix your lists or data paths in the Project Settings cell.")



## 6) Train Recognition Model

This uses `ketos train`. If you already have a base model (e.g., from a previous attempt), put it in `MODELS_DIR` and set its filename below to **optionally continue training**.

- Outputs: intermediate checkpoints + the best model in `MODELS_DIR`.


In [ ]:

import os, shlex, subprocess, glob
from pathlib import Path

# Optional: base model to continue from (leave empty to start from scratch)
BASE_MODEL = ""  # e.g., f"{MODELS_DIR}/attempt_01.mlmodel"

# Where to write the new model
ATTEMPT_NAME = "attempt_01"  # change if you want to keep multiple attempts
OUT_MODEL = f"{MODELS_DIR}/{ATTEMPT_NAME}.mlmodel"

cmd = [
    "ketos","train",
    "-o", OUT_MODEL,
    "--workers", str(int(CPU_THREADS)),
    "--device", DEVICE,
    "-f", "alto",
    TRAIN_LIST, VAL_LIST
]

if BASE_MODEL and Path(BASE_MODEL).exists():
    cmd += ["--load", BASE_MODEL]

print("Running:", " ".join(shlex.quote(x) for x in cmd))
result = subprocess.run(cmd, text=True)

if result.returncode == 0:
    print(f"✅ Training finished. Model at: {OUT_MODEL}")
else:
    raise SystemExit("❌ Training failed. Check the logs above.")



## 7) Evaluate the Model (CER/WER)

Runs `ketos test` on the validation list to get quick metrics.


In [ ]:

import shlex, subprocess

MODEL_FOR_EVAL = OUT_MODEL  # change if you want to evaluate another checkpoint

cmd = ["ketos", "test", "-f", "alto", "-m", MODEL_FOR_EVAL, VAL_LIST]
print("Running:", " ".join(shlex.quote(x) for x in cmd))

res = subprocess.run(cmd, text=True)
if res.returncode == 0:
    print("✅ Evaluation completed. See metrics above.")
else:
    raise SystemExit("❌ Evaluation failed. Check the logs above.")



## 8) (Optional) Run Inference on a Folder

Use your trained model to transcribe a folder of ALTO image/XML pairs. Adjust inputs/outputs as needed.


In [ ]:

# Example inference command; adjust input/output folders to your setup.
# For ALTO pipelines you typically use `ketos` or `kraken` CLI tools depending on your flow.
# Here we just show a pattern; feel free to replace with your exact inference step.

import shlex, subprocess, os
INFERENCE_INPUT = DATA_DIR          # change if needed
INFERENCE_OUTPUT = f"{MODELS_DIR}/inference_out"
os.makedirs(INFERENCE_OUTPUT, exist_ok=True)

print("NOTE: Adjust this cell to your exact inference pipeline if needed.")
print("For ALTO workflows, you may prefer running `kraken` CLI directly for segmentation/recognition.")

# Example placeholder (no-op): list files we would process
!ls -lah "$INFERENCE_INPUT" | head -n 20



## 9) FAQ / Troubleshooting

**Q: Can we install Kraken once on the Colab system forever?**  
A: Colab VMs are **temporary** and reset. We **cannot** persist system installs. The best workaround is what we use here: a **small pip cache on Drive** to make future installs **much faster** without storing a huge environment on Drive.

**Q: I get `No training samples found ...`**  
A: Make sure your `DATA_DIR` really contains image/XML pairs and that the list generator finds them. The sample generator expects **matching filenames** (e.g., `page001.png` with `page001.xml`) and the XML to include `<alto ...>`.

**Q: How do I resume training from a previous checkpoint?**  
A: Put your `.mlmodel` in `MODELS_DIR` and set `BASE_MODEL` to its path. The training cell will add `--load` automatically.

**Q: Can I use GPU?**  
A: Yes. In Colab, go to **Runtime → Change runtime type → T4 GPU**, then set `DEVICE = "cuda"` in Project Settings. If install fails, re-run the Install cell so Torch pulls a CUDA build.
